# Cellular Automaton 2D
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [cellularAutomaton2D.ipynb](https://github.com/diegoinacio/creative-coding-notebooks/blob/master/Simulation/cellularAutomaton2D.ipynb)
---
Implementation of an uni-dimensional cellular automata of binary values.

<font color='#CC0000'>[<b>PT-BR</b> content]</font>

Implementação de um autômato celular bidimensional em valores binários.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as manim
import numpy as np
import imageio

In [ ]:
plt.rcParams['figure.figsize'] = (16, 8)
plt.set_cmap('gray')

## Definição da lista de regra
---
A função ***conjRegras*** resulta em uma lista de regras para definição de novos estados para as células centrais em gerações posteriores, com base em uma vizinhança definida por uma area ou kernel. Se uma célula tem *raio* igual a **1**, seu diâmetro será **3** e sua *area* igual a **9**. Para uma vizinhança de area **9** existem $2^9=512$ padrões possíveis e $2^{512}$ (valor com mais de 150 dígitos) possibilidades de regras.


- Autômato celular com regra **35**

| x                                     | 511     | ...     | 11      | ...     | 5       | ...     | 1       | 0       |
|---------------------------------------|---------|---------|---------|---------|---------|---------|---------|---------|
| **estado atual**                      |111111111| ...     |000001011| ...     |000000101| ...     |000000001|000000000|
| **novo estado para células centrais** | 0       | 0       | 0       | 0       | 1       | 0       | 1       | 1       |

- Autômato celular com regra **2051**

| x                                     | 511     | ...     | 11      | ...     | 5       | ...     | 1       | 0       |
|---------------------------------------|---------|---------|---------|---------|---------|---------|---------|---------|
| **estado atual**                      |111111111| ...     |000001011| ...     |000000101| ...     |000000001|000000000|
| **novo estado para células centrais** | 0       | 0       | 1       | 0       | 0       | 0       | 1       | 1       |

In [ ]:
def conjRegras(regra, area):
    tamanho = 2**area
    bits = list('{0:0{1}b}'.format(regra, tamanho)[::-1])
    resultado = np.array(bits, dtype=np.int)
    return resultado

## Tabela de regras
---
A função ***conjRegras_tab*** define a tabela de regras para visualização.

In [ ]:
def conjRegras_tab(regra, area, disp=(8, 2)):
    '''
    Define a tabela do conjunto de regras
    '''
    raio = int(area**0.5/2)
    diametro = 2*raio + 1
    s, t = disp
    S, T = s*(diametro + 1), t*(diametro + 2)
    crt = np.ones((T, S), np.float32)*0.25
    crList = conjRegras(regra, area)
    total = 2**area
    indices = np.linspace(0, total - 1, s*t, dtype=np.int32)
    for j in range(t):
        for i in range(s):
            try:
                indice = indices[i + j*s]
                bits = list('{0:0{1}b}'.format(indice, area))
                bits = np.array(bits, dtype=np.int).reshape((diametro, diametro))
                iro = (diametro + 1)*i
                iri = iro + diametro
                jro = (diametro + 2)*j
                jri = jro + diametro
                crt[jro:jri, iro:iri] = bits
                crt[jri, iro + raio] = crList[indice]
            except:
                pass
    return crt[:-1, :-1]

## Definição do autômato celular
---
Gera autômato celular a partir da geração 0.

In [ ]:
def automatoCelular(ger, regra, area):
    '''
    Gera autômato celular
    '''
    t, s = ger.shape
    resultado = np.zeros((t, s), np.float32)
    raio = int(area**0.5/2)
    diametro = 2*raio + 1
    novoEstado = conjRegras(regra, area)
    for j, i in np.ndindex(ger.shape):
        roll = [- j + raio, - i + raio]
        vizinhanca = np.roll(ger, roll, axis=(0, 1))[:diametro, :diametro]
        binario = ''.join(map(str, vizinhanca.ravel().astype(int)))
        indice = int(binario, 2)
        resultado[j, i] = novoEstado[indice]
    return resultado

## Autômato
---

In [ ]:
%%time
# Parâmetros
raio = 1
area = (2*raio + 1)**2
tamanho = 2**area
np.random.seed(sum(map(lambda c: ord(c), 'automato celular')))
N = int(tamanho*0.25)
regra = np.array([1]*N + [0]*(tamanho - N))
np.random.shuffle(regra)
regra[0], regra[-1] = 0, 0
regra = int(''.join([str(e) for e in regra]), 2)

# Incialização da geração 0
celulas = (32, 128)
ger0 = np.random.random(celulas) > 0.99

## Visualização
---
### Visualização do autômato
---

In [ ]:
%%time
fig, ax = plt.subplots(figsize=(2048//72, 512//72))

ger1 = ger0.copy()
def animation(frame):
    plt.cla()
    gen = frame
    imshow = ax.imshow(ger1, interpolation='none')
    ger1[:] = automatoCelular(ger1, regra, area)
    ax.set_title('Cellular Automaton 2D | generation {:04}'.format(gen), fontsize=24)
    plt.tight_layout()
    return imshow

anim = manim.FuncAnimation(fig, animation, frames=64, interval=100)

anim.save('output/NLDC_cellularAutomaton2D_anim.gif', writer="imagemagick", extra_args="convert")
plt.close()

# Solve repetition problem
! magick convert _output/NLDC_cellularAutomaton2D_anim.gif -loop 0 _output/NLDC_cellularAutomaton2D_anim.gif
! echo GIF exported and reconverted. Disregard the message above.

![cellular automaton 2D](output/NLDC_cellularAutomaton2D_anim.gif "Cellular Automaton 2D")

### Visualização da tabela de regra
---

In [ ]:
crt = conjRegras_tab(regra, area)
plt.imshow(crt, interpolation='none')
plt.axis('off')
titulo = 'rule = {}'.format(regra)
plt.title(titulo, fontsize=8)
plt.show()

## Exemplos
---
### autômato celular A
---

In [ ]:
%%time
# Parâmetros
raio = 1
area = (2*raio + 1)**2
tamanho = 2**area
np.random.seed(sum(map(lambda c: ord(c), 'automato celular A 123')))
N = int(tamanho*0.123)
regra = np.array([1]*N + [0]*(tamanho - N))
np.random.shuffle(regra)
regra[0], regra[-1] = 0, 0
regra = int(''.join([str(e) for e in regra]), 2)

# Incialização da geração 0
celulas = (32, 128)
ger0 = np.random.random(celulas) > 0.99

fig, ax = plt.subplots(figsize=(2048//72, 512//72))

ger1 = ger0.copy()
def animation(frame):
    plt.cla()
    gen = frame
    imshow = ax.imshow(ger1, interpolation='none')
    ger1[:] = automatoCelular(ger1, regra, area)
    ax.set_title('Cellular Automaton 2D A | generation {:04}'.format(gen), fontsize=24)
    plt.tight_layout()
    return imshow

anim = manim.FuncAnimation(fig, animation, frames=64, interval=100)

anim.save('output/NLDC_cellularAutomaton2D_animA.gif', writer="imagemagick", extra_args="convert")
plt.close()

# Solve repetition problem
! magick convert _output/NLDC_cellularAutomaton2D_animA.gif -loop 0 _output/NLDC_cellularAutomaton2D_animA.gif
! echo GIF exported and reconverted. Disregard the message above.

![cellular automaton 2D A](output/NLDC_cellularAutomaton2D_animA.gif "Cellular Automaton 2D A")

### autômato celular B
---

In [ ]:
%%time
# Parâmetros
raio = 1
area = (2*raio + 1)**2
tamanho = 2**area
np.random.seed(sum(map(lambda c: ord(c), 'automato celular B 234')))
N = int(tamanho*0.234)
regra = np.array([1]*N + [0]*(tamanho - N))
np.random.shuffle(regra)
regra[0], regra[-1] = 0, 0
regra = int(''.join([str(e) for e in regra]), 2)

# Incialização da geração 0
celulas = (32, 128)
ger0 = np.random.random(celulas) > 0.99

fig, ax = plt.subplots(figsize=(2048//72, 512//72))

ger1 = ger0.copy()
def animation(frame):
    plt.cla()
    gen = frame
    imshow = ax.imshow(ger1, interpolation='none')
    ger1[:] = automatoCelular(ger1, regra, area)
    ax.set_title('Cellular Automaton 2D B | generation {:04}'.format(gen), fontsize=24)
    plt.tight_layout()
    return imshow

anim = manim.FuncAnimation(fig, animation, frames=64, interval=100)

anim.save('output/NLDC_cellularAutomaton2D_animB.gif', writer="imagemagick", extra_args="convert")
plt.close()

# Solve repetition problem
! magick convert _output/NLDC_cellularAutomaton2D_animB.gif -loop 0 _output/NLDC_cellularAutomaton2D_animB.gif
! echo GIF exported and reconverted. Disregard the message above.

![cellular automaton 2D B](output/NLDC_cellularAutomaton2D_animB.gif "Cellular Automaton 2D B")

### autômato celular C
---

In [ ]:
%%time
# Parâmetros
raio = 1
area = (2*raio + 1)**2
tamanho = 2**area
np.random.seed(sum(map(lambda c: ord(c), 'automato celular C 345')))
N = int(tamanho*0.345)
regra = np.array([1]*N + [0]*(tamanho - N))
np.random.shuffle(regra)
regra[0], regra[-1] = 0, 0
regra = int(''.join([str(e) for e in regra]), 2)

# Incialização da geração 0
celulas = (32, 128)
ger0 = np.random.random(celulas) > 0.99

fig, ax = plt.subplots(figsize=(2048//72, 512//72))

ger1 = ger0.copy()
def animation(frame):
    plt.cla()
    gen = frame
    imshow = ax.imshow(ger1, interpolation='none')
    ger1[:] = automatoCelular(ger1, regra, area)
    ax.set_title('Cellular Automaton 2D C | generation {:04}'.format(gen), fontsize=24)
    plt.tight_layout()
    return imshow

anim = manim.FuncAnimation(fig, animation, frames=64, interval=100)

anim.save('output/NLDC_cellularAutomaton2D_animC.gif', writer="imagemagick", extra_args="convert")
plt.close()

# Solve repetition problem
! magick convert _output/NLDC_cellularAutomaton2D_animC.gif -loop 0 _output/NLDC_cellularAutomaton2D_animC.gif
! echo GIF exported and reconverted. Disregard the message above.

![cellular automaton 2D C](output/NLDC_cellularAutomaton2D_animC.gif "Cellular Automaton 2D C")